In [1]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install efficientnet

     |████████████████████████████████| 51kB 3.7MB/s 


In [3]:
import datetime
import os
import numpy as np
import tensorflow as tf

from efficientnet.keras import EfficientNetB7
from keras import backend as K

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Dropout, LeakyReLU, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [4]:
np.random.seed = 42

main_path = f"/content/gdrive/My Drive"

dataset = "ISIC 2018"

sub_path = f"{main_path}/Data/{dataset}/256/"

In [5]:
X_train = np.load(f"{sub_path}train_images.npz")["arr_0"]
Y_train = np.load(f"{sub_path}train_labels.npz")["arr_0"]

X_train = X_train / 255
Y_train = Y_train / 255

In [6]:
print(np.max(X_train))
print(np.min(X_train))
print(np.max(Y_train))
print(np.min(Y_train))

1.0
0.0
1.0
0.0


In [7]:
X_train.shape

(2075, 256, 256, 3)

In [8]:
_, h, w, c = X_train.shape

input_shape = (h, w, c)

In [9]:
def Conv_block(x, filters, a=0.01, dr=0.05):
    y = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x)
    y = BatchNormalization(axis=-1)(y)
    y = LeakyReLU(alpha=a)(y)
    y = Dropout(rate=dr)(y)

    return y

In [10]:
def R2_block(x, filters, a=0.01, dr=0.05, depth=3):
    x1 = Conv_block(x, filters, a, dr)

    xn = Conv_block(x1, filters, a, dr)
    cn = Concatenate(axis=-1)([x1, xn])

    for _ in range(depth - 1):
        xn = Conv_block(cn, filters, a, dr)
        cn = Concatenate(axis=-1)([x1, xn])

    return cn

In [11]:
def efficientB7R2Unet(input_shape):
    a = 0.01
    dr = 0.1
    depth = 4

    Encoder = EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 1
    input = Encoder.input
    en1 = R2_block(input, 32, a, dr, depth)

    en1 = Conv_block(en1, 32, a, dr)

    # contracting level 2
    en2 = Encoder.layers[49].output

    # contracting level 3
    en3 = Encoder.layers[152].output

    # contracting level 4
    en4 = Encoder.layers[255].output

    # middle level 5
    en5 = Encoder.layers[551].output
    cv5 = R2_block(en5, 224, a, dr, depth)
    up5 = Conv2DTranspose(224, (2, 2), strides=(2, 2), padding="same")(cv5)

    # expandsion level 4
    cv4 = Concatenate(axis=-1)([up5, en4])
    cv4 = R2_block(cv4, 80, a, dr, depth)
    up4 = Conv2DTranspose(80, (2, 2), strides=(2, 2), padding="same")(cv4)

    # expandsion level 3
    cv3 = Concatenate(axis=-1)([up4, en3])
    cv3 = R2_block(cv3, 48, a, dr, depth)
    up3 = Conv2DTranspose(48, (2, 2), strides=(2, 2), padding="same")(cv3)

    # expandsion level 2
    cv2 = Concatenate(axis=-1)([up3, en2])
    cv2 = R2_block(cv2, 32, a, dr, depth)
    up2 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(cv2)

    # expandsion level 1
    cv1 = Concatenate(axis=-1)([up2, en1])
    cv1 = R2_block(cv1, 32, a, dr, depth)

    cv1 = Conv_block(cv1, 32, a, dr)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv1)

    model = tf.keras.Model(inputs=[input], outputs=[out])

    model.compile(optimizer=Adam(0.01), loss="binary_crossentropy", metrics=["acc", "mse"])

    return model

In [12]:
B = 7
blocks = "R2"

model_name = f"EffnetB{B}_{blocks}"

In [13]:
# model = efficientB7R2Unet(input_shape)

In [14]:
model = tf.keras.models.load_model(f"{main_path}/NN Models/{dataset}/model_ISIC 2018_EffnetB7_R2_checkpoint")

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 64) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [16]:
checkpoint = ModelCheckpoint(filepath=f"{main_path}/NN Models/{dataset}/checkpoint/", monitor="loss", save_best_only=True, save_freq="epoch")

reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.3, patience=5, verbose=1, cooldown=10, min_lr=1e-6)


def scheduler(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * tf.math.exp(-0.01)

lr_shceduler = LearningRateScheduler(scheduler, verbose=1)

In [17]:
K.set_value(model.optimizer.learning_rate, 0.00001)

In [18]:
model.optimizer.learning_rate

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=1e-05>

In [19]:
model.fit(x=X_train, y=Y_train, batch_size=8, epochs=500, verbose=2, callbacks=[checkpoint, reduce_lr, lr_shceduler])

Epoch 1/500

Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-06.
260/260 - 169s - loss: 0.0259 - acc: 0.9821 - mse: 0.0065
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/NN Models/ISIC 2018/checkpoint/assets
Epoch 2/500

Epoch 00002: LearningRateScheduler reducing learning rate to tf.Tensor(9.900497e-06, shape=(), dtype=float32).
260/260 - 136s - loss: 0.0258 - acc: 0.9822 - mse: 0.0065
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/NN Models/ISIC 2018/checkpoint/assets
Epoch 3/500

Epoch 00003: LearningRateScheduler reducing learning rate to tf.Tensor(9.8019855e-06, shape=(), dtype=float32).
260/260 - 136s - loss: 0.0260 - acc: 0.9821 - mse: 0.0066
Epoch 4/500

Epoch 00004: LearningRateScheduler reducing learning rate to tf.Tensor(9.704454e-06, shape=(), dtype=float32).
260/260 - 136s - loss: 0.0261 - acc: 0.9821 - mse: 0.0066
Epoch 5/500

Epoch 00005: LearningRateScheduler reducing learning rate to tf.Tensor(9.607893e-06, shap

KeyboardInterrupt: ignored

In [20]:
model.save(f"{main_path}/NN Models/{dataset}/model_{dataset}_{model_name}_{str(datetime.datetime.now())[:-7]}")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/NN Models/ISIC 2018/model_ISIC 2018_EffnetB7_R2_2021-02-22 20:47:34/assets


In [ ]:
print("\n-----------------------------------------------End of process-----------------------------------------------")